In [2]:
import ml_mmpa
import pandas as pd

In [8]:
# Import data

test_data = pd.read_csv('test_data/test_data.csv')

In [7]:
# Your data should have two columns: 'SMILES', 'Property'

test_data.head()

,Smiles,pMIC
0,CCCCCCCCCCCCCCCCCC(=O)NC(CCCN)C(=O)NC(CCCN)C(=...,4.99
1,CC1CN(c2c(F)cc3c(=O)c(C(=O)O)cn(C4CC4)c3c2Cl)CCN1,7.18
2,CN[C@@H]1[C@@H](O)[C@@H](O[C@@H]2[C@@H](O)[C@H...,5.44
3,CCCCCCCC(=O)N[C@@H](C(=O)N[C@H](CCN)C(=O)N(C)C...,4.48
4,O=C(O)c1cn(C2CC2)c2c(cc(F)c3nn(CN4CCOCC4)c(=S)...,6.53


In [9]:
# carry out mmpa
# clean mmpa
# prodcue functions for mmpa
# summarise results in a table

### MMPA